In [ ]:
!pip install -U -q folium
import folium
import numpy as np
import pandas as pd
import seaborn as sns
import geopy.distance
from collections import Counter
import matplotlib.pyplot as plt
from IPython.display import display
from scripts.cogsci_module import *
import warnings
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')
%matplotlib inline

# [COGSCI 1B] Typology
---
### Professor Terry Regier

This module explores a central question in cognitive science and linguistics: how do languages vary from one another? We will explore datasets of linguistic features ([PHOIBLE](http://phoible.org/) and [WALS](http://wals.info/)) to come to tentative answers to this question in a data-driven way. Example problems include visualizing the distribution of phonemes, the relationship between geography and the development of languages, and the genetic relationships of languages.

---

### Table of Contents

0 - [PHOIBLE Data](#PHOIBLE Data)

1 - [Phoneme Distributions](#phoneme dist)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1.1 - [Consonants](#consonants)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1.2 - [Vowels](#vowels)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1.3 - [Phonemes](#phonemes)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1.4 - [Consonants vs Vowels](#cons vs vows)<br>

2 - [Phoneme Metadata](#metadata)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.1 - [Family](#phoneme fam)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.2 - [Continent](#phoneme cont)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.3 - [Latitude and Longitude](#lat lons)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.4 - [Population Size vs Phoneme Inventory Size](#pop v foam)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.5 - [Distance from Africa](#africa distance)

3 - [Common Phonemes](#common)

4 - [Tone](#tone)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4.1 - [Altitude](#altitude)

5 - [Morphological Complexity](#morph complex)


---

## PHOIBLE Data<a id='PHOIBLE Data'></a>

We will start by familiarizing ourselves with the data and at the same time learn some common ways of analyzing data in the Python programming language. In order to do that, we need to load them into our notebook. 

First, we'll start with our data from [PHOIBLE](http://phoible.org/). In the code cell below, we create a variable called `file_name` that we assign to the name of our file in quotations. Note that we have `phoible_data/` in front of the file name, which means that our file `phoible_elevation.csv` is in the `phoible_data` directory (folder). We turn this file into what is called a [**DataFrame**](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html), which can be thought of as a slightly more rigid Excel sheet. It allows us to easily access, manipulate, and visualize our data.

A code cell will display what is written in the last line of the cell (if it is not a variable assignment statement). So in the cell below, the last line says `phoible_data.head()`, which means that it will display our dataframe, but adding `.head()` at the end of it allows us to show only the first 5 rows.

In [ ]:
file_name = 'phoible_data/phoible_elevation.csv'  # assign the file path to a variable
phoible_data = pd.read_csv(file_name)  # read in the CSV to pandas
phoible_data.head()  # print the first five rows

In our dataframe, the column `Population` was stored as *strings*, not numbers, because some values in the column are words. As *string* is essentially text for the computer, and it can't perform all of the same mathematical operations on characters of text. The possible text entries we find in the `Population` column for those rows are shown below.

In [ ]:
sorted(list(set(phoible_data['Population'])))[-5:]

In order to use the numerical values of `Population` for further analysis, we are going to drop rows where the values are words, and convert the numbers to be represented as `ints`, or the Python representation of integers, and create a new dataframe called `phoib` with this new data. Reasons like this emphasize the importance of being aware of how your data is represented and how you store data.

In [ ]:
# phoib contains rows where population is a number
phoib = phoible_data.copy()
phoib["Population"] = pd.to_numeric(phoib['Population'], errors='coerce')
phoib = phoib.dropna(subset=['Population'])  # drop non integer rows
length_difference = len(phoible_data) - len(phoib)
print("When we remove those rows with text, we lose {} rows.".format(length_difference))
phoib.head()

The next thing that we'll notice is that there are multiple rows for some of the languages. The cell below counts how many times a value in the column `LanguageCode` occurs.

In [ ]:
phoible_data['LanguageCode'].value_counts().head(10)

That's weird, we might expect each language code to have a unique language name. Let's look at all of the rows where the `LanguageCode` is equal to `gwn`.

In [ ]:
# same language code, but different language name.
cond = phoible_data['LanguageCode'] == 'gwn'
phoible_data[cond]

It appears that the six rows for `gwn` are the same language, just different dialects. There are cases like this, and also cases where the same language has an entry from more than one source (Korean is an example of one). Below is the number of different possible language codes, and below that, the total number of rows.

In [ ]:
print('There are {} unique languages in phoible_data'.format(len(list(set(phoible_data['LanguageCode'])))))

In [ ]:
print('There are {} total rows in phoible_data'.format(len(phoible_data)))

In the following cell, we are going to map each of the rows from the `phoible_data` table based on their values in the `Latitude` and `Longitude` columns in our DataFrame. Don't worry if it takes a while, there are over 2000 points that have to be plotted!

In [ ]:
mp = folium.Map(zoom_start=12)
phoib_coords = phoible_data.dropna(subset=['Latitude', 'Longitude'])
for coords in list(zip(phoib_coords['Latitude'], phoib_coords['Longitude'])):
    folium.Circle(
        radius=100,
        location=coords,
        color='crimson',
        fill=False,).add_to(mp)
mp

---

# Phonemic Inventories

Phonemes are individual sounds. They come from the [IPA](https://en.wikipedia.org/wiki/International_Phonetic_Alphabet), which is the phonetic alphabet. Phonemes can either be consonants or vowels. Languages have a fixed number of phonemes -- and the best data source for this is PHOIBLE, which we loaded above.

One big question in cognitive science is determining the relationship between linguistic features (e.g. number of vowels, word order and number of tense categories) and non-linguistic features (e.g. population size, altitude and climate). In particular, a lot of attention has been paid to the relationship between population size and various linguistic features. For example, people have looked at the relationship between population size and the size of the phonemic inventory, and population size and morphological complexity.

## Phoneme Distributions <a id='phoneme dist'></a>

Below we are going to plot the numerical distributions of the number of phonemes a language has (broken into consonants, vowels, and total phonemes), as well as the geographic distributions of those values. We'll be using histograms to visualize the numerical distributions and plot on a map for the geographic spread.

### Consonants <a id='consonants'></a>

We are going to first average together the values that each dialect has to get a row for each language code. Therefore, we'll use the `groupby` method followed by the `mean` to average the dialects:

In [ ]:
phoib_avg = phoib.groupby('LanguageCode').mean().iloc[:,-7:].reset_index()
phoib_avg.head()

In [ ]:
sns.distplot(phoib_avg['Consonants'])
plt.ylabel('proportion per consonant')

What do you notice about the distribution of consonants?

The following function splits the values that it gets for consonants into five different color categories and plots them on a map. Again, this might take a few seconds!

In [ ]:
map_with_bins('Consonants', phoib_avg.dropna())

You'll notice that most of the dots are dark blue, except for some in southern Africa. The bins for this method are determined by taking the range from the smallest value to the largest value, then dividing that range into five evenly-sized sections. Looking back at our histogram, you'll see that almost all of our data falls into the two blue bins (values less than 55.6). So in this situation, it is hard to get much out of the visualization.

To combat that, we'll divide the bins based on quantiles. That is, we'll split the bins up so that each bin contains the same number of points. The lowest fifth of values in the dark-blue bin, the second lowest fifth in the light-blue, and so on. By passing the phrase `quantiles=True` into the function, we are able to do that. Take notice of the ranges of values in each bin, and compare those to the map above.

In [ ]:
# plotting with bins set on quantiles instead
map_with_bins('Consonants', phoib_avg.dropna(), quantiles=True)

What do you notice?

### Vowels <a id='vowels'></a>

We can repeat the above for `Vowels`, go ahead and write some code to bin and map the vowel distributions.

First use `sns.distplot` for our `phoib_avg` dataframe for `Vowels`:

In [ ]:
# task

How does this distribution differ from the Consonant distribution?

Now use the `map_with_bins` function to bin and map:

In [ ]:
# task

Are there any areas where the colors seem significantly different between the maps for vowels and consonants?

### Phonemes <a id='phonemes'></a>

And the same for total `Phonemes`.

Get the distribution with `sns.distplot`:

In [ ]:
# task

And now `map_with_bins`:

In [ ]:
# task

---

### Consonants vs Vowels <a id='cons vs vows'></a>

Next, we will visualize the relationship between number of consonants and number of vowels. The following graph plots a line-of-best-fit through the data. If you want to read more about how that line is determined, look [here](https://www.inferentialthinking.com/chapters/14/1/regression-model.html).

In [ ]:
overlay_hex(phoib_avg["Consonants"], phoib_avg["Vowels"])

What can you infer about the relationsihp between the number of consonants and the number of vowels, based purely on the line? How accurate do you think the line is? Can we trust it?

Below we calculate the average number of consonants per vowel that each **dialect** (note that we are using `phoib` instead of `phoib_avg`) has for a continent / geographical region.

In [ ]:
pho_cont = phoib[["Area","Consonants", "Vowels"]].copy()
pho_cont = pho_cont.groupby(by="Area").mean()
pho_cont['Ratio'] = pho_cont['Consonants'] / pho_cont['Vowels']
pho_cont[['Ratio']].plot.bar(figsize = (12,8))
plt.title('Average Consonants per Vowel')

What effect could the choice to calculate these values using dialects instead of the grouped language codes have on the outcomes?

---

## Phoneme Metadata <a id='metadata'></a>

### By Family  <a id='phoneme fam'></a>

We are now going to take a look at the effect that some of those non-linguistic features have on the linguistic features.

In the next cell, we import our [WALS](http://wals.info/) data. There aren't any immediate problems with the data set that we have to address.

In [ ]:
wals = pd.read_csv('wals_data/language.csv')
wals.head()

Currently, we have genetic affilitation data in the column `family` in WALS, and we have the `area` column we saw previously in PHOIBLE, as well as many of the linguistic features. To do some of our analysis, we are first going to need to join our two dataframes together. WALS and PHOIBLE both identify languages with ISO 639-3, so we are able to pair rows together with that information. We will call our new dataframe `combined`.

In [ ]:
combined = wals.dropna(subset=['iso_code']).merge(phoib.dropna(), left_on='iso_code', right_on='LanguageCode', how='inner')
combined.head()

Our first visualization is a bar plot of the average number of total phonemes, consonants, and vowels for each genetic family.

In [ ]:
print('There are {} different families.'.format(len(combined.family.value_counts())))

In [ ]:
# double click on the image to zoom in (then you can scroll left or right)
combined.groupby(by="family")[['Phonemes', 'Consonants', 'Vowels']].mean().plot.bar(figsize=(50,8))
plt.title('Mean Number of __ for Each Family')
plt.ylabel('Count')

There are a lot of different families so to get a zoomed in version, double click on the plot.

### By Continent  <a id='phoneme cont'></a>

We will recreate the same chart as above, except this time we will group together by `area` instead of `family`.

In [ ]:
phoib.groupby(by="Area")[['Phonemes', 'Consonants', 'Vowels']].mean().plot.bar(figsize=(12,6))
plt.title('Mean Number of __ for Each Area')
plt.ylabel('Count')

### Latitude and Longitude <a id='lat lons'></a>

Next, we'll look at the number of phonemes plotted against latitude and longitude.

In [ ]:
sns.jointplot('Phonemes', 'Latitude', data=phoib, kind='hex')
plt.title('Phonemes vs Latitude')

Using the syntax in the cell above, plot the `Phonemes` and `Longitude` below:

In [ ]:
# task

Are you able to make any meaningful conclusions from these two graphs?

### Population Size vs. Phoneme Inventory Size <a id='pop v foam'></a>

Below we visualize the relationship between population size and phoneme inventory size.

In [ ]:
overlay_hex(phoib["Phonemes"], phoib["Population"])

To get a better picture of what is going on, let's take the log of population. There are more in-depth reasons as to why you might want to take the log of a column before working with it, which you can read about [here](https://stats.stackexchange.com/questions/18844/when-and-why-should-you-take-the-log-of-a-distribution-of-numbers).

Follow the syntax of the cell above, but use `np.log` around `phoib["Population"]` to visualize the log population:

In [ ]:
# task

How can you interpret the above graph?

### Distance from Africa <a id='africa distance'></a>

Someone has claimed that phoneme inventory size and distance from Africa are inversely related. We will test that.

We start by refering back to a subset of a graph we previously created, and look at the average number of phonemes per dialect for each area.

In [ ]:
pho_pop_cont = phoib.loc[:,["Area", "Phonemes"]]
pho_pop_cont = pho_pop_cont.groupby(by = "Area").mean().sort_values('Phonemes', ascending=False)
pho_pop_cont[['Phonemes']].plot.bar(figsize = (12,8))
plt.ylabel('Average Number of Phonemes')
pho_pop_cont

Based on that bar chart, are you able to make any inferences about what will show up when we plot the distance to each individual dialect's distance to Africa and its relation to the number of phonemes?  

To calculate the distance from each point to Africa, we've chose the latitude/longitude pair `8.7832, 34.5085` to be our center point of the continent, and will calculate the distance to each dialect's listed location (in kilometers). We then plot the distance and phoneme values against each other.

In [ ]:
coordinates = list(zip(phoib.dropna()['Latitude'], phoib.dropna()['Longitude']))

# chose this point b/c it comes up when
# you google search 'africa coordinates'
africa_center = (8.7832, 34.5085)

# calculate the distance to each language's listed location
distances = np.array([geopy.distance.vincenty(point, africa_center).km for point in coordinates])

overlay_hex(distances, phoib.dropna()['Phonemes'])
plt.xlabel('Kilometers from Africa')

---

## Common Phenomes <a id='common'></a>

What are the most common phonemes in the world? What is the distribution of frequency? There are a couple thousand phonemes in PHOIBLE, but only a handful are common and there's a long tail. 

Our previous Phoible data set contained mainly metadata and aggregated numbers. We will now pull in a second Phoible data set that contains a row for each phoneme that a language (and possibly a dialect) have.

In [ ]:
phonemes = pd.read_csv('phoible_data/phoible-by-phoneme.tsv', delimiter='\t')
phonemes.head()

In [ ]:
print('Out of {} rows, there are {} unique phonemes.'.format(len(phonemes), len(list(set(phonemes['Phoneme'])))))

In the cell below, we construct a dataframe that counts the number of times that a phoneme occurs in `phonemes`.

In [ ]:
phoneme_counts = pd.DataFrame.from_dict(Counter(phonemes['Phoneme']), orient='index').reset_index().sort_values(0, ascending=False)
phoneme_counts.columns = ['Phoneme', 'Count']
phoneme_counts.head(10)

We then plot the number of times that the top 200 phonemes occur in that table.

In [ ]:
phoneme_counts.iloc[:200].plot.bar(figsize=(15, 5))
plt.xticks([])
plt.ylabel('Count')
plt.xlabel('Phoneme')
plt.title('Counts of 200 Most Common Phonemes')

You can see that there is a quick drop-off in the frequency of different phonemes, and that there is a long right tail. Let's use a histogram to look at the distribution of these frequencies.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
sns.distplot(phoneme_counts['Count'], ax=ax)
plt.title('Distribution of Phoneme Frequency')

What does this the histogram tell us about the frequency of different phonemes?

Phonemes can be described by a set of (mostly) binary features. PHOIBLE has this data too. Is the distribution of featue values evenly split for each feature? If not, which features are more prone to being either 0 or 1? Are some phonemes only present in some area of genetic affiliation?

In [ ]:
lc_to_area = dict(zip(phoib['LanguageCode'], phoib['Area']))

def convert_code(code):
    try:
        return lc_to_area[code]
    except:
        return 'undefined'
    
phonemes['Area'] = [convert_code(code) for code in phonemes['LanguageCode']]

pd.crosstab(phonemes['Phoneme'], phonemes['Area'])

In [ ]:
# normalizing by columns means that it accounts for the fact that
# there are differing numbers of languages per country
pd.crosstab(phonemes['Phoneme'], phonemes['Area'], margins=True, normalize='columns')

---

## Tone <a id='tone'></a>

One of the features of phonemes is `tone`. If a language has a phoneme with tone, it counts as a "tone language". Are most language tone? Where are the tone languages on the map?

In [ ]:
tone_languages = phoib_avg['Tones'] > 0
num_tone_languages = sum(tone_languages)
total_languages = len(phoib_avg)

print('There are {} tone languages out of our dataset of {} languages.'.format(num_tone_languages, total_languages))
print("That's about {}%.".format(np.round(num_tone_languages/total_languages*100, 2)))

In order to plot the tone languages, we create a table of just tone languages.

In [ ]:
tone = phoib_avg[tone_languages]
tone.head()

Then we plot them.

In [ ]:
mappable_tone = tone.dropna(subset=['Latitude', 'Longitude'])

mp = folium.Map(zoom_start=12)
for coords in list(zip(mappable_tone['Latitude'], mappable_tone['Longitude'])):
    folium.Circle(
        radius=100,
        location=coords,
        color='crimson',
        fill=False,).add_to(mp)
mp

What do you notice about the locations of tone languages? How does it compare to the first map we created, where we mapped allow of the dialects?

### Altitude <a id='altitude'></a>

Someone has claimed there is a relationship between being a tone language and the altitude. We've collected altitude from Google Maps Elevation based on the list coordinates for each language. 

We'll ignore the rows that do not have an elevation.

In [ ]:
phoib_avg['Tone Language?'] = phoib_avg['Tones'] > 0
have_elevation = phoib_avg[['elevation', 'Tones', 'Tone Language?']].dropna()

We will plot the histograms of elevation below for languages who are a tone language (in orange) and those that aren't (in blue).

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
sns.distplot(have_elevation[np.invert(have_elevation['Tone Language?'])]['elevation'], ax=ax)
sns.distplot(have_elevation[have_elevation['Tone Language?']]['elevation'], ax=ax)

Is there a difference between the elevation distributions for the two?

The are some pretty significant outliers that are affecting our data. Below, we'll recreate the above visualization, except this time, not including values that are three standard deviations outside of the mean.

In [ ]:
# getting rid of the 3 SD outliers to get a better picture
no_out=have_elevation[((have_elevation['elevation'] - have_elevation['elevation'].mean()) / have_elevation['elevation'].std()).abs() < 3]

f, ax = plt.subplots(figsize=(10, 8))
sns.distplot(no_out[np.invert(no_out['Tone Language?'])]['elevation'], ax=ax)
sns.distplot(no_out[no_out['Tone Language?']]['elevation'], ax=ax)

---

## Relationship between population size and morphological complexity <a id='morph complex'></a>

All the data for this part comes from WALS. Morphological complexity is a vague term, referring to how complicated the words in a language are. Below we look at some features in relation to phoneme inventory size.

### Feature 30A: Number of Genders

In [ ]:
desired_columns = ['LanguageCode', 'Area', 'Latitude', 'Longitude', 'Population', 'Phonemes']

gender_data = drop_and_subset('30A Number of Genders', combined, desired_columns)
genders_dict = {'1 None':1, '2 Two':2, '3 Three':3, '4 Four':4, '5 Five or more':5}
gender_data['Genders'] = [genders_dict[value] for value in gender_data['30A Number of Genders']]

print('Rows with Gender data: {}'.format(len(gender_data)))
gender_data.head()

In [ ]:
overlay_hex(gender_data['Genders'], gender_data['Phonemes'])

In [ ]:
overlay_hex(gender_data['Genders'], np.log(gender_data['Population']))

### Reduplication

In [ ]:
reduplication = drop_and_subset('27A Reduplication', combined, desired_columns)
reduplication.groupby('27A Reduplication').mean()[['Phonemes']].plot.bar()
plt.xticks(rotation=40)

### Feature 20A: Fusion of Selected Inflectional Formatives

In [ ]:
fusion = drop_and_subset('20A Fusion of Selected Inflectional Formatives', combined, desired_columns)
fusion.groupby('20A Fusion of Selected Inflectional Formatives').mean()[['Phonemes']].plot.bar()
plt.xticks(rotation=70)

### Feature 21A: Exponence of Selected Inflectional Formatives

In [ ]:
exponence_a = drop_and_subset('21A Exponence of Selected Inflectional Formatives', combined, desired_columns)
exponence_a.groupby('21A Exponence of Selected Inflectional Formatives').mean()[['Phonemes']].plot.bar()
plt.xticks(rotation=70)

### Feature 21B: Exponence of Tense-Aspect-Mood Inflection

In [ ]:
exponence_b = drop_and_subset('21B Exponence of Tense-Aspect-Mood Inflection', combined, desired_columns)
exponence_b.groupby('21B Exponence of Tense-Aspect-Mood Inflection').mean()[['Phonemes']].plot.bar()
plt.xticks(rotation=70)